In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http

In [4]:
import pandas as pd

In [21]:
farm = pd.DataFrame()
farm['Price'] = [632541, 425618, 356471, 7412512]
farm['Rooms'] = [2, 5, 3, 100]
farm['Square_Feet'] = [1600, 2850, 1780, 90000]
print(farm)

     Price  Rooms  Square_Feet
0   632541      2         1600
1   425618      5         2850
2   356471      3         1780
3  7412512    100        90000


In [22]:
df=spark.createDataFrame(farm);
df.show()

+-------+-----+-----------+
|  Price|Rooms|Square_Feet|
+-------+-----+-----------+
| 632541|    2|       1600|
| 425618|    5|       2850|
| 356471|    3|       1780|
|7412512|  100|      90000|
+-------+-----+-----------+



In [23]:
ms=df.columns;
input_columns_Number_Type=[];
input_columns_String_Type=[];
for i in ms:
  if str(df.schema[i].dataType)=='StringType':
    input_columns_String_Type.append(i);
  else:
    input_columns_Number_Type.append(i);

In [24]:
input_columns_String_Type

[]

In [25]:
input_columns_Number_Type

['Price', 'Rooms', 'Square_Feet']

# **Computing percentiles for all Numeric Column**

In [26]:
d={};
for c in input_columns_Number_Type:
  d[c]=df.approxQuantile(c,[0.01,0.99],0.25);

In [27]:
d

{'Price': [356471.0, 7412512.0],
 'Rooms': [2.0, 100.0],
 'Square_Feet': [1600.0, 90000.0]}

# **Check skewness for all numeric columns**

In [28]:
from pyspark.sql.functions import *

In [31]:
for col in input_columns_Number_Type:
  skew_value_for_the_column=df.select(skewness(c).alias("sk")).collect()[0].asDict()['sk'];
  if skew_value_for_the_column>1:
    df=df.withColumn(col,log(when(df[col]<d[col][0],d[col][0]).when(df[col]>d[col][1],d[col][1]).otherwise(df[col])+1));
  elif skew_value_for_the_column<-1:
    df=df.withColumn(col,exp(when(df[col]<d[col][0],d[col][0]).when(df[col]>d[col][1],d[col][1]).otherwise(df[col])));

In [32]:
df.show()

+------------------+------------------+------------------+
|             Price|             Rooms|       Square_Feet|
+------------------+------------------+------------------+
|12.784010974377479|1.0986122886681098|7.3783837129967145|
|12.784010974377479|1.0986122886681098|7.3783837129967145|
|12.784010974377479|1.0986122886681098|7.3783837129967145|
|12.784010974377479|1.7254630513334037|7.3783837129967145|
+------------------+------------------+------------------+

